# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Action Points

## 1. No need to make 10 transactions a day. Just one is enough
## 2. See the lema gap size on the losing months
## 3. Can I use actual tick value (close) instead of lema for moves?

In [2]:
%%time
data = run_engine(data)

100%|██████████████████████████████████| 40821/40821 [00:17<00:00, 2358.89it/s]


pl    0.0611
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
39499 2022-07-14 09:50:00         NaN  sema_close -0.00143
39570 2022-07-14 15:45:00         NaN  sema_close -0.00286
39598 2022-07-14 18:05:00         NaN  sema_close -0.00036
39599 2022-07-14 18:10:00         NaN  sema_close -0.00010
39600 2022-07-14 18:15:00         NaN  sema_close -0.00010
39601 2022-07-14 18:20:00         NaN  sema_close -0.00010
40029 2022-07-18 06:00:00         NaN  sema_close  0.00168
40190 2022-07-18 19:25:00         NaN  sema_close  0.00126
40473 2022-07-19 19:00:00         NaN  sema_close  0.00788
40609 2022-07-20 06:20:00         NaN  sema_close -0.00050
40632 2022-07-20 08:15:00         NaN  sema_close -0.00069
40633 2022-07-20 08:20:00         NaN  sema_close -0.00010
40634 2022-07-20 08:25:00         NaN  sema_close -0.00010
40659 2022-07-20 10:30:00         NaN  sema_close -0.00059
40660 2022-07-20 10:35:00         NaN  sema_clos

100%|███████████████████████████████████████| 512/512 [00:00<00:00, 875.19it/s]

Wall time: 3min 13s


In [3]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

month_val
1    0.01852
2   -0.01755
3    0.02408
4    0.01944
5    0.01274
6   -0.01573
7    0.01960
Name: pl, dtype: float64
---------------------
max : 3 : 0.02408
min : 2 : -0.01755
---------------------


In [4]:
data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
1,0.002,19,35.0,-16.0,0.07685,-0.04858,0.02827
0,0.001,22,37.0,-15.0,0.08124,-0.05849,0.02275
2,0.004,9,23.0,-14.0,0.04508,-0.03991,0.00517
3,0.003,5,18.0,-13.0,0.02292,-0.02775,-0.00483
6,0.008,1,6.0,-5.0,0.00134,-0.00622,-0.00488
4,0.005,7,12.0,-5.0,0.03494,-0.02915,0.00579
5,0.007,2,6.0,-4.0,0.00258,-0.00538,-0.00280
8,0.009,1,4.0,-3.0,0.00252,-0.00875,-0.00623
9,0.010,1,2.0,-1.0,0.00561,-0.00833,-0.00272
7,0.006,5,5.0,0.0,0.02501,-0.00332,0.02169


In [5]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [6]:
sys.exit()

SystemExit: 

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
df_small

In [ ]:
order_side = 'positive'
y = pd.DataFrame(data['df_small'][data['df_small']['pl_type'] == order_side].lema_gap.round(3).value_counts())
y= y.reset_index()
y = y.rename(columns = {'lema_gap':order_side})

order_side = 'negative'
x= pd.DataFrame(data['df_small'][data['df_small']['pl_type'] == order_side].lema_gap.round(3).value_counts())
x = x.reset_index()
x = x.rename(columns = {'lema_gap':order_side})

all = x.merge(y, on = 'index', how = 'outer')
all = all.fillna(0)

all['diff'] = all['positive'] - all['negative']
all = all.sort_values(by = ['diff', 'index'])
all

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [ ]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
data['plot_start']  = f"{data['input_year']}-12-1"
data['plot_stop']   = f"{data['input_year']}-12-1"
# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']

plot_graph(data)

In [ ]:
x = 252
data['df_ohlc']['lema_angle'][x-3:x+3]

generate_result_report(data)